## Import

In [9]:
from skimage.segmentation import slic
import os
import radiomics
from radiomics import featureextractor
import numpy as np
import nrrd
import os
import numpy as np
import nrrd
import pandas as pd
from radiomics import featureextractor
from skimage.util import view_as_blocks
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import csv


import logging
# ... or set level for specific class
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)


## Funzioni Utili 2D

In [10]:

# Funzione per caricare il volume e la maschera
def carica_volume(file_path):
    return nrrd.read(file_path)[0]

def estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx):
    if not np.any(slice_maschera > 0):
        return None, None
    
    try:
        # Converti le slice da NumPy a SimpleITK
        sitk_slice_immagine = numpy_to_sitk(slice_immagine)
        sitk_slice_maschera = numpy_to_sitk(slice_maschera)

        # Estrai le features usando l'estrattore di SimpleITK
        results = estrattore.execute(sitk_slice_immagine, sitk_slice_maschera)
        
        # Trasforma i risultati in un vettore di features
        feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                                   for key, value in results.items() 
                                   if isinstance(value, (float, int, np.ndarray))])
        return feature_vector, results
    except Exception as e:
        print(f"Errore slice {idx}: {e}")  # Stampa l'errore specifico
        return None, None  # Non aggiungere la slice al CSV

# Funzione per creare il CSV con i nomi delle features
def crea_csv_features(pazienti_dati, features_name, csv_file):
    if not pazienti_dati:
        return

    headers = ['Paziente', 'Slice'] + features_name  # Nomi delle features

    with open(csv_file, mode='w', newline='') as file_csv:
        writer = csv.writer(file_csv)
        writer.writerow(headers)  # Scrivi l'intestazione

        for paziente, slice_idx, feature_vector in pazienti_dati:
            writer.writerow([paziente, slice_idx] + list(feature_vector))  # Scrivi i dati di ogni paziente


def numpy_to_sitk(array_numpy):
    array_numpy = np.expand_dims(array_numpy, axis = -1)
    return sitk.GetImageFromArray(array_numpy)


# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None


## Radiomica 2D

In [ ]:
import csv
import numpy as np
import SimpleITK as sitk
import nrrd
from radiomics import featureextractor
from radiomics import getFeatureClasses

print(list(getFeatureClasses().keys()))

# Percorsi delle directory
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Lista per i dati dei pazienti
pazienti_dati = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor(force2D=True, force2Ddimension=2)
print("Impostazioni Estrattore:", estrattore.settings)

# Disabilita tutte le feature
estrattore.disableAllFeatures()

# Abilita tutte le categorie di feature eccetto shape3D
estrattore.enableFeatureClassByName('firstorder')
estrattore.enableFeatureClassByName('glcm')
estrattore.enableFeatureClassByName('gldm')
estrattore.enableFeatureClassByName('glrlm')
estrattore.enableFeatureClassByName('glszm')
estrattore.enableFeatureClassByName('ngtdm')

# Disabilita le feature wavelet
estrattore.enableImageTypeByName('Wavelet')

# Abilita le feature shape2D
estrattore.enableFeatureClassByName('shape2D')

print("Feature Types Abilitati:", estrattore.enabledImagetypes)

for paziente_folder in sorted(os.listdir(pazienti_dir)):
    paziente_path = os.path.join(pazienti_dir, paziente_folder)
    roi_path = os.path.join(roi_dir, paziente_folder)

    print(paziente_folder)

    if os.path.isdir(paziente_path) and os.path.isdir(roi_path):
        file_immagine_nrrd = trova_nrrd_file(paziente_path)
        file_maschera_nrrd = trova_nrrd_file(roi_path)
        
        if file_immagine_nrrd and file_maschera_nrrd:
            volume_immagine = carica_volume(file_immagine_nrrd)
            volume_maschera = carica_volume(file_maschera_nrrd)
            
            #print(volume_immagine.shape)
            #print(volume_maschera.shape)

            if volume_immagine.shape != volume_maschera.shape:
                print(f"Il volume dell'immagine e la maschera non hanno la stessa forma per il paziente {paziente_folder}")
                continue
            
            # Itera ogni slice
            for idx in range(volume_immagine.shape[2]):
                slice_immagine = volume_immagine[:, :, idx]
                slice_maschera = volume_maschera[:, :, idx]

                if np.any(slice_maschera):  # Verifica se ci sono pixel non nulli
                    feature_vector, results = estrai_features_slice(slice_immagine, slice_maschera, estrattore, idx)
                else:
                    #print(f"Saltato {idx}")
                    continue
                if feature_vector is not None:
                    features_name = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]
                    pazienti_dati.append((paziente_folder, idx + 1, feature_vector))  # paziente_folder come identificativo paziente

# Ordina i dati prima di creare il CSV
pazienti_dati.sort(key=lambda x: (int(x[0]), x[1]))  # Ordina per paziente e poi per slice

# Crea il CSV con i dati estratti
crea_csv_features(pazienti_dati, features_name, 'C:\\Users\\bsbar\\Desktop\\Radiomica_Wavelet_25D.csv')

## Combinazione Radiomica + Rete Pretrainata

In [10]:
import pandas as pd

# Percorsi dei file CSV
path_primo_csv = r"C:\Users\bsbar\Desktop\Radiomica_Wavelet_2D.csv"
path_secondo_csv = r"C:\Users\bsbar\Desktop\INCEPTION_ALL_SLICES_with_3D_indices.csv"
path_output_csv = r"C:\Users\bsbar\Desktop\Rad_Inc_2D_Combinato_Wavelet_pre.csv"

# Apri i due file CSV
csv1 = pd.read_csv(path_primo_csv)
csv2 = pd.read_csv(path_secondo_csv)

# Verifica che le colonne "Paziente" e "Slice" esistano nel primo CSV
if 'Paziente' not in csv1.columns or 'Slice' not in csv1.columns:
    raise ValueError("Le colonne 'Paziente' e 'Slice' non sono presenti nel primo CSV.")

# Verifica che le colonne "Patient" e "Slice" esistano nel secondo CSV
if 'Patient' not in csv2.columns or 'Slice' not in csv2.columns:
    raise ValueError("Le colonne 'Patient' e 'Slice' non sono presenti nel secondo CSV.")

# Unisci i due DataFrame sui campi "Paziente" (del primo CSV) e "Patient" (del secondo CSV), e "Slice"
csv_combinato = pd.merge(csv1, csv2, left_on=['Paziente', 'Slice'], right_on=['Patient', 'Slice'], how='left')

# Rimuovi la colonna "Patient" duplicata
csv_combinato.drop(columns=['Patient'], inplace=True)

# Salva il nuovo CSV
csv_combinato.to_csv(path_output_csv, index=False)

print(f"Operazione completata. Il nuovo file è stato salvato in {path_output_csv}.")


Operazione completata. Il nuovo file è stato salvato in C:\Users\bsbar\Desktop\Rad_Inc_2D_Combinato_Wavelet_pre.csv.


## Rimozioni Colonne Inutili

In [11]:
# Carica il file CSV
file_path = r"C:\Users\bsbar\Desktop\Rad_Inc_2D_Combinato_Wavelet_pre.csv" # Sostituisci con il percorso del tuo file
df = pd.read_csv(file_path)

# Filtra le colonne che non iniziano con "diagnostic" e "3D_Slice_Index"
df_filtered = df[[col for col in df.columns if not (col.startswith('diagnostic') or col.startswith('3D_Slice_Index'))]]

# Specifica il percorso per salvare il file filtrato
output_path = r"C:\Users\bsbar\Desktop\Rad_Inc_2D_Combinato_Wavelet.csv"  # Sostituisci con il percorso dove vuoi salvare il file

# Salva il nuovo file CSV
df_filtered.to_csv(output_path, index=False)

print(f"Le colonne sono state rimosse e il nuovo file è stato salvato come '{output_path}'.")

Le colonne sono state rimosse e il nuovo file è stato salvato come 'C:\Users\bsbar\Desktop\Rad_Inc_2D_Combinato_Wavelet.csv'.


## Features Radiomiche 3D

In [4]:
# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
dimension_mismatch = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per verificare le dimensioni delle immagini
def verifica_dimensioni(immagine_file, maschera_file):
    immagine, _ = nrrd.read(immagine_file)
    maschera, _ = nrrd.read(maschera_file)
    return immagine.shape == maschera.shape

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    print(paziente)
    
    # Verifica se la cartella esiste anche per le ROI
    roi_paziente_dir = os.path.join(roi_dir, paziente)
    paziente_dir = os.path.join(pazienti_dir, paziente)

    if os.path.isdir(roi_paziente_dir):
        # Trova i file immagine e maschera nella rispettiva cartella
        immagine_file = trova_nrrd_file(paziente_dir)
        maschera_file = trova_nrrd_file(roi_paziente_dir)
     
        if immagine_file and maschera_file:
            # Verifica se le dimensioni delle immagini e delle ROI corrispondono
            if verifica_dimensioni(immagine_file, maschera_file):
                # Aggiungi il nome del paziente alla lista
                patient_names.append(paziente)

                # Estrai le features radiomiche
                results = estrattore.execute(immagine_file, maschera_file)

                # Trasforma i risultati in un vettore di features
                feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                           for key, value in results.items() 
                           if isinstance(value, (float, int, np.ndarray))])
                
                # Aggiungi il vettore delle features alla lista
                feature_vectors.append(feature_vector)
                
            else:
                # Aggiungi il nome del paziente e un messaggio di errore alla lista delle dimensioni non corrispondenti
                dimension_mismatch.append(paziente)

# Trasforma la lista di vettori in un array numpy
feature_matrix = np.array(feature_vectors)

# Ora hai:
# - 'feature_matrix': una matrice dove ogni riga è il vettore di features di un paziente
# - 'patient_names': una lista dei nomi dei pazienti corrispondente
# - 'dimension_mismatch': una lista dei nomi dei pazienti con dimensioni non corrispondenti

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
12
120
123
124
126
127
128
129
133
135
136
137
138
139
141
142
144
146
147
149
15
150
153
155
158
159
16
161
163
166
168
169
17
170
171
175
176
178
182
183
188
189
19
190
193
197
199
200
205
22
23
24
25
26
27
29
30
31
33
35
36
38
39
40
41
42
43
44
46
47
48
5
50
52
53
54
56
57
58
59
60
61
62
64
65
68
69
70
71
74
75
76
78
79
81
82
84
85
86
87
88
89
90
91
92
94
95
96
98


In [5]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'Radiomica_3D.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

File CSV salvato come Radiomica_3D.csv


In [36]:
import radiomics
from radiomics import featureextractor, getFeatureClasses

# Ottieni le classi di feature disponibili
feature_classes = getFeatureClasses()
print("Feature classes disponibili:")
for feature_class_name in feature_classes:
    print(feature_class_name)

Feature classes disponibili:
firstorder
glcm
gldm
glrlm
glszm
ngtdm
shape
shape2D


In [3]:
# Funzione per visualizzare una sezione specifica di un blocco 3D
def visualizza_blocco(blocco, sezione=None):
    if sezione is None:
        sezione = blocco.shape[2] // 2  # Visualizza la sezione centrale per default
    
    # Visualizza la sezione specifica del blocco
    plt.imshow(blocco[:, :, 11], cmap='gray')
    plt.title(f"Blocco - Sezione {sezione}")
    plt.axis('off')  # Nascondi gli assi
    plt.show()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per caricare il volume 3D delle immagini
def carica_volume_paziente(cartella_paziente):
    sezioni = []  # Lista per conservare le singole sezioni 2D
    for file in sorted(os.listdir(cartella_paziente)):
        if file.endswith('.nrrd'):
            # Leggi la sezione 2D
            immagine, _ = nrrd.read(os.path.join(cartella_paziente, file))
            sezioni.append(immagine)  # Aggiungi la sezione alla lista
    # Stack delle sezioni per formare un volume 3D (asse delle sezioni = asse z)
    volume_3d = np.stack(sezioni, axis=-1)
    return volume_3d

# Funzione per suddividere il volume in blocchi 3D
def suddividi_volume(volume_3d, dimensione_blocco=(16, 16, 16)):
    # Verifica se le dimensioni del volume sono divisibili esattamente per il blocco
    shape_volume = np.array(volume_3d.shape)
    if np.any(shape_volume % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni del volume non sono divisibili esattamente per il blocco 3D.")
    
    # Usa view_as_blocks per suddividere il volume in blocchi 3D
    blocchi = view_as_blocks(volume_3d, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in una lista di blocchi singoli
    blocchi_risagomati = blocchi.reshape(-1, *dimensione_blocco)
    return blocchi_risagomati

# Funzione per convertire un array NumPy in un oggetto SimpleITK
def numpy_to_sitk(volume_numpy):
    return sitk.GetImageFromArray(volume_numpy)

# Funzione per verificare se la maschera ha valori non nulli
def verifica_maschera_valida(maschera):
    return np.any(maschera > 0)


def riduci_slice(volume_3d, target_slices=512):
    slice_correnti = volume_3d.shape[-1]
    
    if slice_correnti < target_slices:
        slice_da_aggiungere = target_slices - slice_correnti
        slice_inizio = slice_da_aggiungere // 2
        slice_fine = slice_da_aggiungere - slice_inizio
        padding = ((0, 0), (0, 0), (slice_inizio, slice_fine))
        volume_pad = np.pad(volume_3d, pad_width=padding, mode='constant', constant_values=0)
        return volume_pad

    elif slice_correnti > target_slices:
        slice_da_rimuovere = slice_correnti - target_slices
        slice_inizio = slice_da_rimuovere // 2
        slice_fine = slice_da_rimuovere - slice_inizio
        volume_ridotto = volume_3d[:, :, slice_inizio:slice_correnti-slice_fine]
        return volume_ridotto

    else:
        return volume_3d

# Funzione per suddividere sia la placca che la maschera
def suddividi_placca_maschera(volume_3d, maschera, dimensione_blocco=(32, 32, 32)):
    # Applica la maschera per estrarre solo la parte della placca
    placca = np.where(maschera > 0, volume_3d, 0)
    
    # Verifica se le dimensioni della placca e della maschera sono divisibili esattamente per il blocco
    shape_placca = np.array(placca.shape)
    shape_maschera = np.array(maschera.shape)
    if np.any(shape_placca % np.array(dimensione_blocco) != 0) or np.any(shape_maschera % np.array(dimensione_blocco) != 0):
        raise ValueError("Le dimensioni della placca o della maschera non sono divisibili esattamente per il blocco 3D.")
    
    # Suddividi in blocchi sia la placca che la maschera
    blocchi_placca = view_as_blocks(placca, block_shape=dimensione_blocco)
    blocchi_maschera = view_as_blocks(maschera, block_shape=dimensione_blocco)
    
    # Riorganizza i blocchi in liste di blocchi singoli
    blocchi_placca_risagomati = blocchi_placca.reshape(-1, *dimensione_blocco)
    blocchi_maschera_risagomati = blocchi_maschera.reshape(-1, *dimensione_blocco)
    
    return blocchi_placca_risagomati, blocchi_maschera_risagomati